In [ ]:
import pandas as pd
from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import Html2TextTransformer

In [ ]:
input_df = pd.read_excel(r"C:\Users\Dell\Downloads\Input for Name collection model _20231027.xlsx")

In [ ]:
input_df = input_df.dropna(subset='Link')

In [ ]:
len(input_df['Link'].unique())

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-ner-english")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/roberta-large-ner-english")


##### Process text sample (from wikipedia)

from transformers import pipeline

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [ ]:

def get_text(url):
    print("fetching url ", url)
    urls = [url]
    loader = AsyncHtmlLoader(urls)
    docs = loader.load()
    html2text = Html2TextTransformer()
    docs_transformed = html2text.transform_documents(docs)
    return docs_transformed

In [ ]:
name_list = []

def get_name(cont,url):
    print("getting_names....")
    for i in cont.split("\n\n"):

        a = nlp(i)
        if a:
            for ent in a:
                if ent['entity_group'] == 'PER':
                    name = ent['word']
                    data = {
                        "Names":name,
                        "Links":url
                    }
                    name_list.append(data)

In [ ]:
for url in input_df['Link'].unique():
#url = input_df['Link'].unique()[0]
    print(url)
    docs_transformed = get_text(url)
    cont = docs_transformed[0].page_content.lower()
    get_name(cont,url)
#docs_transformed[0].metadata['source']
print("--completed--")



In [ ]:
import pandas as pd
df = pd.DataFrame(name_list)

In [ ]:
df['Names'] = df['Names'].str.lower()

In [ ]:
df['Names'] = df['Names'].str.strip()

In [ ]:
len(df)

In [ ]:
df_unique = df.drop_duplicates()

In [ ]:
df_unique = df_unique.reset_index(drop=True)

In [ ]:
final_list = []
for i,j in zip(df_unique['Names'],df_unique['Links']):
    if not len(i.split()) < 2:
        final_list.append({
            "name":i,
            "link":j
        })

In [ ]:
out_df = pd.DataFrame(final_list)

In [ ]:
out_df.to_excel("E:\\name_idendified.xlsx", index = False)